![Piggy bank](piggy_bank.jpg)

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to `boolean` data type:<br> `1` if `"yes"`, otherwise `0` |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0` |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type:<br> `1` if `"success"`, otherwise `0`. |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0`. |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

In [51]:
import pandas as pd
import numpy as np

# Start coding here...

In [52]:
df = pd.read_csv("bank_marketing.csv")

for col in ["credit_default", "mortgage", "previous_outcome", "campaign_outcome"]:
    print(col)
    print("--------------")
    print(df[col].value_counts())

credit_default
--------------
no         32588
unknown     8597
yes            3
Name: credit_default, dtype: int64
mortgage
--------------
yes        21576
no         18622
unknown      990
Name: mortgage, dtype: int64
previous_outcome
--------------
nonexistent    35563
failure         4252
success         1373
Name: previous_outcome, dtype: int64
campaign_outcome
--------------
no     36548
yes     4640
Name: campaign_outcome, dtype: int64


In [53]:
print(df.head())
print(df.describe())
print(df.info())

   client_id  age  ... euribor_three_months campaign_outcome
0          0   56  ...                4.857               no
1          1   57  ...                4.857               no
2          2   37  ...                4.857               no
3          3   40  ...                4.857               no
4          4   56  ...                4.857               no

[5 rows x 16 columns]
         client_id          age  ...  cons_price_idx  euribor_three_months
count  41188.00000  41188.00000  ...    41188.000000          41188.000000
mean   20593.50000     40.02406  ...       93.575664              3.621291
std    11890.09578     10.42125  ...        0.578840              1.734447
min        0.00000     17.00000  ...       92.201000              0.634000
25%    10296.75000     32.00000  ...       93.075000              1.344000
50%    20593.50000     38.00000  ...       93.749000              4.857000
75%    30890.25000     47.00000  ...       93.994000              4.961000
max    4118

In [54]:
clients_ds = df[["client_id", "age", "job", "marital", "education", "credit_default", "mortgage"]].copy()
campaings_ds = df[["client_id", "number_contacts", "contact_duration", "previous_campaign_contacts", "previous_outcome", "campaign_outcome"]]
economics_ds = df[["client_id", "cons_price_idx", "euribor_three_months"]]
print(clients_ds.head())
print(clients_ds.info())
print(campaings_ds.head())
print(campaings_ds.info())
print(economics_ds.head())
print(economics_ds.info())

   client_id  age        job  marital    education credit_default mortgage
0          0   56  housemaid  married     basic.4y             no       no
1          1   57   services  married  high.school        unknown       no
2          2   37   services  married  high.school             no      yes
3          3   40     admin.  married     basic.6y             no       no
4          4   56   services  married  high.school             no       no
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   client_id       41188 non-null  int64 
 1   age             41188 non-null  int64 
 2   job             41188 non-null  object
 3   marital         41188 non-null  object
 4   education       41188 non-null  object
 5   credit_default  41188 non-null  object
 6   mortgage        41188 non-null  object
dtypes: int64(2), object(5)
memory usage: 2.2+

In [55]:
clients_ds["job"] = clients_ds["job"].str.replace(".", "_")
clients_ds["education"] = clients_ds["education"].str.replace(".", "_")
clients_ds["education"].replace('unknown', np.nan, inplace=True)

try:
    clients_ds["credit_default"] = (clients_ds["credit_default"].str.lower() == "yes")
    clients_ds["mortgage"] = (clients_ds["mortgage"].str.lower() == "yes")
except:
    print("Already converted")

print(clients_ds.head())
print(clients_ds.info())
print(clients_ds['mortgage'].describe())

   client_id  age        job  marital    education  credit_default  mortgage
0          0   56  housemaid  married     basic_4y           False     False
1          1   57   services  married  high_school           False     False
2          2   37   services  married  high_school           False      True
3          3   40     admin_  married     basic_6y           False     False
4          4   56   services  married  high_school           False     False
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   client_id       41188 non-null  int64 
 1   age             41188 non-null  int64 
 2   job             41188 non-null  object
 3   marital         41188 non-null  object
 4   education       39457 non-null  object
 5   credit_default  41188 non-null  bool  
 6   mortgage        41188 non-null  bool  
dtypes: bool(2), int64(2), object(

In [56]:
try:
    campaings_ds["previous_outcome"] = (campaings_ds["previous_outcome"].str.lower() == "success")
    campaings_ds["campaign_outcome"] = (campaings_ds["campaign_outcome"].str.lower() == "yes")
except:
    print("Already converted")

months_dict = {'jan':'01', 'feb':'02', 'mar':'03', 'apr':'04', 'may':'05', 'jun':'06', 'jul':'07', 'aug':'08', 'sep':'09', 'oct': '10', 'nov': '11', 'dec':'12'}


df["year"] = 2022
campaings_ds["last_contact_date"] = df["year"].astype(str) + "-" + df["month"].astype(str).map(months_dict) + "-" + df["day"].apply(lambda x: f"{int(x):02}")

print(df["month"].unique())
print(campaings_ds.head())
print(campaings_ds.info())
print(campaings_ds['previous_outcome'].describe())

['may' 'jun' 'jul' 'aug' 'oct' 'nov' 'dec' 'mar' 'apr' 'sep']
   client_id  number_contacts  ...  campaign_outcome  last_contact_date
0          0                1  ...             False         2022-05-13
1          1                1  ...             False         2022-05-19
2          2                1  ...             False         2022-05-23
3          3                1  ...             False         2022-05-27
4          4                1  ...             False         2022-05-03

[5 rows x 7 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 7 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   client_id                   41188 non-null  int64 
 1   number_contacts             41188 non-null  int64 
 2   contact_duration            41188 non-null  int64 
 3   previous_campaign_contacts  41188 non-null  int64 
 4   previous_outcome            41188 n

In [57]:
clients_ds.to_csv("client.csv", index=False)
campaings_ds.to_csv("campaign.csv", index=False)
economics_ds.to_csv("economics.csv", index=False)